# Customer Service Requests Analysis

### 1) Understanding the dataset

Importing required libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import f_oneway

cwd = os.getcwd()

Dataframe creation/Importing dataset

In [ ]:
df_service = pd.read_csv(cwd + '/311_Service_Requests_from_2010_to_Present.csv', header = 0)

Visualizing the dataset

In [ ]:
df_service.info()

Printing the columns of the DataFrame

In [ ]:
print(df_service.columns)

Identifying the shape of the dataset

In [ ]:
# identifying number of rows and columns
df_service.shape

Identifying the variables with null values

In [ ]:
df_service.isnull().sum()

In [ ]:
# Dropping columns that would not contribute to the Analysis
drop_cols = ['Intersection Street 1','Intersection Street 2',
             'Address Type','Landmark','Incident Address','Street Name','Cross Street 1','Cross Street 2']
df_service.drop(drop_cols, inplace=True, axis=1)
df_service.info()

### 2) Performing basic data exploratory analysis

Drawing a frequency plot to show the number of null values in each column of the DataFrame

In [ ]:
# Using the value_counts() here because value_counts() already counts frequency and putting it in a bar graph
df_service_null = df_service.isnull().sum()
df_service_null[df_service_null > 0].value_counts().plot.barh()
plt.title('Number of null values in each column')
plt.xlabel('Frequency')
plt.ylabel('Null Count')

Missing value treatment

In [ ]:
# Removing the records whose Closed Date values are null
df_service.dropna(inplace=True,subset=['Closed Date'])
df_service.isnull().sum()

In [ ]:
df_service.info()

Analyzing the date column and removing entries that have an incorrect timeline

In [ ]:
# Converting the dates to datetime format
df_service['Created Date'] = pd.to_datetime(df_service['Created Date'], errors='coerce')
df_service['Closed Date'] = pd.to_datetime(df_service['Closed Date'], errors='coerce')

In [ ]:
# Calculating the time elapsed in closed and creation date
df_service['Elapsed Time'] = df_service['Closed Date'] - df_service['Created Date']

In [ ]:
# Converting the calculated date(elapsed time) to seconds and minutes to get a better representation
df_service['Elapsed Time Seconds'] = (df_service['Elapsed Time']).dt.total_seconds()
df_service['Elapsed Time Minutes'] = df_service['Elapsed Time']/np.timedelta64(1,'m')


In [ ]:
# Descriptive statistics for the newly created column
df_service['Elapsed Time Seconds'].describe()

In [ ]:
# Number of null values in the Complaint_Type column
df_service['Complaint Type'].isnull().sum()

In [ ]:
# Number of null values in the City column
df_service['City'].isnull().sum()

In [ ]:
# Impute the NaN value with 'Unknown City'
df_service['City'].fillna(inplace=True,value='Unknown City')
df_service['City'].isnull().sum()

In [ ]:
# Frequency plot for the complaints in each city
plt.figure().set_size_inches(15,5)
df_service['City'].value_counts().plot.bar()
plt.title('Number of complaints in each city')
plt.xlabel('City')
plt.ylabel('Complaints')


In [ ]:
# Analysis of compliants across Brooklyn
df_brooklyn = df_service[df_service['Borough']=='BROOKLYN']
df_brooklyn.info()

In [ ]:
# Scatter and Hexbin plot of the concentration of complaints across Brooklyn
df_brooklyn.plot(kind='scatter', title = 'Compliants concentration across Brooklyn', 
                 figsize=(18,9), x='Longitude', y='Latitude', alpha=0.1)
df_brooklyn.plot(kind='hexbin', title = 'Compliants concentration across Brooklyn', 
                 figsize=(12,6), x='Longitude', y='Latitude', gridsize=30, cmap='inferno', mincnt=1).axis('equal')

### 3) Finding major types of complaints

Bar graph to show the types of complaints

In [ ]:
df_service['Complaint Type'].value_counts().plot(kind='bar', title='Types of Complaints', 
                                                 figsize=(10,4), xlabel='\nComplaint Type', ylabel='Count')

Frequency of various types of complaints for New York City

In [ ]:
df_newyork = df_service[df_service['City']=='NEW YORK']
df_newyork['Complaint Type'].value_counts()

Overall top 10 complaint types

In [ ]:
df_service['Complaint Type'].value_counts().head(10)

Various types of complaints in each city

In [ ]:
df_complaints_each_city = df_service.groupby(['City', 'Complaint Type'])
df_complaints_each_city.size().to_frame('Count').reset_index()

New DataFrame, which contains cities as columns and complaint types in rows

In [ ]:
df_new = df_service[['City', 'Complaint Type']]
#df_new.pivot_table(values='Complaint Type', index=df_new.index, columns='City')
#df_new.reset_index().groupby(['City']['Complaint Type']).aggregate('first').unstack()
df_new['City'].transpose()
df_new

### 4) Visualize the major types of complaints in each city

Chart that shows the types of complaints in each city in a single chart, where different colors show the different types of complaints

In [ ]:
df_pivot = df_service.pivot_table(index='City', 
                                  columns = 'Complaint Type', values = 'Unique Key', aggfunc = 'count')
df_pivot = df_pivot.sort_values(by=['Blocked Driveway'],ascending=False)
df_pivot.iloc[:10,:].plot(kind='bar', figsize=(20,15), 
                          title='Number of Complaints by Type and City', ylabel = 'Count', stacked=True)

Sorting the complaint types based on the average Elapsed Time(Request Closing Time) and grouping them for different locations

In [ ]:
df_service['Elapsed Time'].describe()

In [ ]:
df_locationwise_elapsed_time = df_service.groupby(['Complaint Type','Borough'])[['Elapsed Time Minutes']].mean().unstack()
df_locationwise_elapsed_time.head()



Verifying whether the average response time across different complaint types is similar (overall)

In [ ]:
# Visualizing the average of Elapsed Time in Minutes

df_locationwise_elapsed_time.plot(subplots='Elapsed Time Minutes',
                                  figsize=(20,15), title='Average time taken to resolve a Complaints by Type and City', 
                                  ylabel = 'Avg. Time(in Mins)', stacked=True)

Identifing the significant variables by performing statistical analysis using p-values

In [ ]:
# Since 'Noise' of any type is the most frequent complaint, it is taken for analysis
df_all_noise = df_service[df_service['Complaint Type'].str.contains('Noise')]['Elapsed Time Minutes']
df_all_noise.hist(range=(0,2000)) # The data is left skewed, needs to be converted to gaussian

In [ ]:
# performing the same analysis as above on the compliant type 'Blocked Driveway' for comparison
df_blk_driveway = df_service[df_service['Complaint Type']=='Blocked Driveway']['Elapsed Time Minutes']
df_blk_driveway.hist(range=(0,2000))


In [ ]:
# New Dataset
log_dataset={}
for i in df_service['Complaint Type'].unique():
    log_dataset[i] = np.log(df_service[df_service['Complaint Type']==i]['Elapsed Time Minutes'])
log_dataset.keys()

In [ ]:
# ['Noise - Street/Sidewalk', 'Noise - Commercial', 'Noise - House of Worship', 'Noise - Vehicle',  'Noise - Park']
log_dataset['Noise - Street/Sidewalk'].hist()

In [ ]:
log_dataset['Blocked Driveway'].hist()

### ANOVA Analysis

In [ ]:
# Oneway ANOVA Analysis (Checking for all Noise complaints)
# Null Hypothesis: The average response time across complaint types is not different
# Alternate Hypothesis: The average response time across complaint types is different

f_oneway(log_dataset['Noise - Street/Sidewalk'], log_dataset['Noise - Commercial'], 
log_dataset['Noise - House of Worship'], log_dataset['Noise - Vehicle'], log_dataset['Noise - Park'])


Based on the above dataset (with all Noise related complaints), the p value returned by Oneway ANOVA test is > 0.05, which accepts the Null Hypothesis.

In [ ]:
# Let us now try Oneway ANOVA test with a different set of complaints in the dataset
# This time let us check for the top 5 complaints
f_oneway(log_dataset['Blocked Driveway'],
    log_dataset['Illegal Parking'],
    log_dataset['Noise - Street/Sidewalk'],
    log_dataset['Noise - Commercial'],
    log_dataset['Derelict Vehicle'])

Based on the above dataset (with top 5 complaints), the p value returned by Oneway ANOVA test is <> 0.05, which rejects the Null Hypothesis.

### Kruskal-Wallis H test

Let us now run the Kruskal Wallis test on the same datasets.

In [ ]:
# Kruskal Wallis test (Checking for all Noise complaints)
stats.kruskal(log_dataset['Noise - Street/Sidewalk'], log_dataset['Noise - Commercial'], 
log_dataset['Noise - House of Worship'], log_dataset['Noise - Vehicle'], log_dataset['Noise - Park'])
         

Based on the above dataset (with all Noise related complaints), the p value returned by Oneway ANOVA test is > 0.05, which accepts the Null Hypothesis.

Both oneway and kruskal wallis test produce similar results.

In [ ]:
stats.kruskal(log_dataset['Blocked Driveway'],
    log_dataset['Illegal Parking'],
    log_dataset['Noise - Street/Sidewalk'],
    log_dataset['Noise - Commercial'],
    log_dataset['Derelict Vehicle'])

Based on the above dataset (with all Noise related complaints), the p value returned by Oneway ANOVA test is < 0.05, which rejects the Null Hypothesis.

Both oneway and kruskal wallis test produce similar results.